# Lab 00g: Working with APIs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab00g_working_with_apis.ipynb)

Learn to make HTTP requests, handle JSON responses, and work with security APIs.

## Learning Objectives
- Make HTTP requests using Python's `requests` library
- Parse JSON responses into Python dictionaries
- Handle API errors gracefully
- Work with API keys securely
- Understand rate limiting and pagination

## Why This Matters for Security

Modern security tools rely heavily on APIs:

| API Type | Use Case | Examples |
|----------|----------|----------|
| **Threat Intel** | Check IP/domain reputation | VirusTotal, AbuseIPDB, Shodan |
| **SIEM/SOAR** | Query logs and alerts | Splunk, Elastic, Microsoft Sentinel |
| **LLM** | AI-powered analysis | Anthropic, OpenAI, Google |
| **Ticketing** | Create/update incidents | ServiceNow, Jira |

In [ ]:
#@title Install dependencies (Colab only)
#@markdown Run this cell to install required packages in Colab

%pip install -q requests python-dotenv

In [ ]:
# Import libraries
import requests
import json
import os
import time
from typing import Optional, Dict, Any, List

print("✅ Libraries loaded!")

## 1. HTTP Basics

### The Request-Response Cycle

```
┌─────────────────────────────────────────────────────────────┐
│                    HTTP REQUEST/RESPONSE                     │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   Your Code                        API Server               │
│   ─────────                        ──────────               │
│                                                             │
│   1. Request ─────────────────────►                         │
│      GET /api/v1/ip/8.8.8.8                                 │
│      Headers: Authorization: Bearer xxx                     │
│                                                             │
│   2. Response ◄────────────────────                         │
│      Status: 200 OK                                         │
│      Body: {"ip": "8.8.8.8", "malicious": false}           │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### HTTP Methods Quick Reference

| Method | Purpose | Example |
|--------|---------|---------|
| **GET** | Retrieve data | Get IP reputation |
| **POST** | Send data | Submit file for scanning |
| **PUT** | Update data | Update alert status |
| **DELETE** | Remove data | Delete old records |

### Status Codes Quick Reference

| Code | Meaning | What To Do |
|------|---------|------------|
| 200 | Success | Parse the response |
| 400 | Bad Request | Check your parameters |
| 401 | Unauthorized | Check your API key |
| 403 | Forbidden | You don't have permission |
| 404 | Not Found | Resource doesn't exist |
| 429 | Rate Limited | Slow down, wait and retry |
| 500 | Server Error | API is having issues |

## 2. Making Your First GET Request

Let's query a public API to get information about an IP address.

**No API key needed for these practice APIs!**

In [ ]:
# Basic GET request - Query IP information
# This API returns geographic info about an IP address

url = "https://ipinfo.io/8.8.8.8/json"

# Make the request
response = requests.get(url)

# Check if successful
print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    # Parse JSON response into Python dictionary
    data = response.json()
    print(f"\n📍 IP Info for 8.8.8.8:")
    print(f"   City: {data.get('city', 'Unknown')}")
    print(f"   Region: {data.get('region', 'Unknown')}")
    print(f"   Country: {data.get('country', 'Unknown')}")
    print(f"   Org: {data.get('org', 'Unknown')}")
else:
    print(f"Error: {response.status_code}")

### How to Read This Code

1. **`requests.get(url)`** - Sends HTTP GET request to the URL
2. **`response.status_code`** - HTTP status (200 = success)
3. **`response.json()`** - Converts JSON response to Python dict
4. **`data.get('key', 'default')`** - Safely gets value (returns default if missing)

In [ ]:
# Query parameters - Adding filters to your request
# HTTPBin echoes back what you send - great for testing!

url = "https://httpbin.org/get"

# Add query parameters (becomes: /get?ip=192.168.1.1&action=lookup)
params = {
    "ip": "192.168.1.1",
    "action": "lookup"
}

# Make request with parameters
response = requests.get(url, params=params)

print(f"Full URL: {response.url}")
print(f"Status: {response.status_code}")
print(f"\nResponse (what the server saw):")
print(json.dumps(response.json()["args"], indent=2))

## 3. Adding Headers (Authentication)

Most security APIs require authentication via headers.

```
┌─────────────────────────────────────────────────────────────┐
│                    API AUTHENTICATION                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   Common Header Patterns:                                   │
│                                                             │
│   Authorization: Bearer sk-abc123...      ← Most APIs       │
│   X-API-Key: abc123...                    ← Some APIs       │
│   api-key: abc123...                      ← Azure/Custom    │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Example: Making authenticated request (using httpbin for demo)

url = "https://httpbin.org/headers"

# Headers for authentication
headers = {
    "Authorization": "Bearer your-api-key-here",
    "Content-Type": "application/json",
    "User-Agent": "SecurityBot/1.0"  # Some APIs check this
}

response = requests.get(url, headers=headers)

print("Headers the server received:")
print(json.dumps(response.json()["headers"], indent=2))

## 4. POST Requests - Sending Data

POST requests send data TO the server (like submitting a form or adding an IOC to your threat intel platform).

In [ ]:
# POST request - Send JSON data to server

url = "https://httpbin.org/post"

# Data to send (would be threat intel in real scenario)
payload = {
    "ioc_type": "ip",
    "value": "185.143.223.47",
    "tags": ["malicious", "c2", "cobalt-strike"],
    "confidence": 0.95
}

# POST with JSON body
response = requests.post(
    url,
    json=payload,  # Automatically converts dict to JSON
    headers={"Content-Type": "application/json"}
)

print(f"Status: {response.status_code}")
result = response.json()
print(f"\nServer received this data:")
print(json.dumps(result.get('json'), indent=2))

## 5. Error Handling - Don't Let Your Script Crash!

APIs can fail for many reasons. Always handle errors gracefully.

### Common Failure Modes

| Error | Cause | Solution |
|-------|-------|----------|
| Timeout | Server too slow | Set timeout, retry |
| Connection | Network issue | Retry with backoff |
| 401 | Bad API key | Check credentials |
| 429 | Rate limited | Wait and retry |
| 500 | Server error | Retry later |

In [ ]:
# Safe API call pattern - ALWAYS use this in production!

from typing import Optional, Dict

def safe_api_call(url: str, timeout: int = 10) -> Optional[Dict]:
    """
    Make a safe API call with proper error handling.

    Args:
        url: The API endpoint to call
        timeout: Seconds to wait before timeout

    Returns:
        JSON response as dict, or None if failed
    """
    try:
        # Set timeout to prevent hanging forever
        response = requests.get(url, timeout=timeout)

        # Raise exception for 4xx/5xx status codes
        response.raise_for_status()

        return response.json()

    except requests.exceptions.Timeout:
        print(f"⏱️ Request timed out after {timeout}s")
        return None

    except requests.exceptions.HTTPError as e:
        print(f"❌ HTTP error: {e.response.status_code}")
        return None

    except requests.exceptions.ConnectionError:
        print("🔌 Connection failed - check network")
        return None

    except requests.exceptions.RequestException as e:
        print(f"💥 Request failed: {e}")
        return None

# Test with valid URL
print("Test 1: Valid URL")
result = safe_api_call("https://httpbin.org/json")
if result:
    print(f"✅ Success! Got: {list(result.keys())}")

# Test with invalid URL
print("\nTest 2: Invalid URL")
result = safe_api_call("https://this-url-does-not-exist-12345.com")

# Test with timeout
print("\nTest 3: Slow server (will timeout)")
result = safe_api_call("https://httpbin.org/delay/15", timeout=2)

In [ ]:
# Secure API key loading pattern

from dotenv import load_dotenv

# Load .env file (create one with your keys)
load_dotenv()

def get_api_key(key_name: str) -> Optional[str]:
    """
    Securely load API key from environment.

    Args:
        key_name: Name of environment variable

    Returns:
        The API key or None if not found
    """
    key = os.getenv(key_name)
    if key:
        # Log success without exposing the key
        print(f"✅ {key_name} loaded (length: {len(key)} chars)")
    else:
        print(f"⚠️ {key_name} not found in environment")
    return key

# Example: Try to load various API keys
print("Checking for common security API keys:\n")

keys_to_check = [
    "VIRUSTOTAL_API_KEY",
    "ABUSEIPDB_API_KEY",
    "ANTHROPIC_API_KEY",
    "OPENAI_API_KEY"
]

for key_name in keys_to_check:
    get_api_key(key_name)

### Setting Up Your .env File

Create a file called `.env` in your project root:

```
# .env file (NEVER commit this to git!)
VIRUSTOTAL_API_KEY=your-key-here
ABUSEIPDB_API_KEY=your-key-here
ANTHROPIC_API_KEY=sk-ant-your-key
```

Add to `.gitignore`:
```
.env
*.env
.env.*
```

## 7. Rate Limiting - Be a Good API Citizen

APIs limit requests to prevent abuse. Respect these limits!

```
┌─────────────────────────────────────────────────────────────┐
│                    RATE LIMITING                             │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   Your Script: Request! Request! Request! Request!          │
│                         │                                   │
│                         ▼                                   │
│   API Server: 429 Too Many Requests - SLOW DOWN!           │
│                                                             │
│   Better Approach:                                          │
│   Request → Wait 1s → Request → Wait 1s → Request          │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Rate-limited requests
from typing import List

def rate_limited_requests(
    urls: List[str],
    requests_per_minute: int = 30
) -> List[Optional[Dict]]:
    """
    Make multiple requests with rate limiting.

    Args:
        urls: List of URLs to request
        requests_per_minute: Max requests per minute

    Returns:
        List of responses (None for failed requests)
    """
    delay = 60 / requests_per_minute  # Seconds between requests
    results = []

    for i, url in enumerate(urls):
        print(f"Request {i+1}/{len(urls)}...", end=" ")

        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            results.append(response.json())
            print(f"✅ {response.status_code}")
        except Exception as e:
            results.append(None)
            print(f"❌ {type(e).__name__}")

        # Wait before next request (except for last one)
        if i < len(urls) - 1:
            time.sleep(delay)

    return results

# Demo: Make 3 requests with rate limiting
test_urls = [
    "https://httpbin.org/json",
    "https://httpbin.org/uuid",
    "https://httpbin.org/ip"
]

print("Making rate-limited requests (30/min = 2s delay):\n")
results = rate_limited_requests(test_urls, requests_per_minute=30)
print(f"\n✅ Completed {len([r for r in results if r])} successful requests")

## 8. Building a Security API Client

Let's put it all together and build a reusable security API client!

In [ ]:
# Complete Security API Client

class SecurityAPIClient:
    """
    A robust API client for security operations.

    Features:
    - Automatic error handling
    - Rate limiting
    - Secure key management
    """

    def __init__(self, base_url: str, api_key: Optional[str] = None):
        """
        Initialize the client.

        Args:
            base_url: Base URL for API (e.g., "https://api.example.com")
            api_key: Optional API key for authentication
        """
        self.base_url = base_url.rstrip('/')
        self.api_key = api_key
        self.session = requests.Session()  # Reuse connections for efficiency

        # Set default headers
        self.session.headers.update({
            "User-Agent": "SecurityAPIClient/1.0",
            "Content-Type": "application/json"
        })

        if api_key:
            self.session.headers["Authorization"] = f"Bearer {api_key}"

    def get(self, endpoint: str, params: Optional[Dict] = None) -> Optional[Dict]:
        """
        Make GET request with error handling.

        Args:
            endpoint: API endpoint (e.g., "/v1/ip/check")
            params: Optional query parameters

        Returns:
            JSON response or None if failed
        """
        url = f"{self.base_url}{endpoint}"

        try:
            response = self.session.get(url, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"❌ GET {endpoint} failed: {e}")
            return None

    def post(self, endpoint: str, data: Dict) -> Optional[Dict]:
        """
        Make POST request with error handling.

        Args:
            endpoint: API endpoint
            data: JSON data to send

        Returns:
            JSON response or None if failed
        """
        url = f"{self.base_url}{endpoint}"

        try:
            response = self.session.post(url, json=data, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"❌ POST {endpoint} failed: {e}")
            return None

# Demo with httpbin
print("🔧 Testing SecurityAPIClient\n")

client = SecurityAPIClient("https://httpbin.org")

# GET request
result = client.get("/get", params={"ip": "192.168.1.1"})
if result:
    print(f"✅ GET successful - received args: {result.get('args')}")

# POST request
result = client.post("/post", data={"threat": "malware", "severity": "high"})
if result:
    print(f"✅ POST successful - server received: {result.get('json')}")

## 🎉 Congratulations!

You now know how to:

1. **Make HTTP requests** with `requests` library
2. **Parse JSON** responses into Python dicts
3. **Handle errors** gracefully with try-except
4. **Manage API keys** securely with environment variables
5. **Respect rate limits** with delays and retries

## Key Takeaways

| Concept | Code Pattern |
|---------|-------------|
| GET request | `requests.get(url, params=params)` |
| POST request | `requests.post(url, json=data)` |
| Parse JSON | `response.json()` |
| Add headers | `headers={"Authorization": "Bearer key"}` |
| Handle errors | `try: ... except RequestException:` |
| Rate limit | `time.sleep(delay)` between requests |

## Next Steps

Now you're ready for:
- **Lab 04**: Use LLM APIs for log analysis
- **Lab 05**: Build an agent with threat intel API tools
- **Lab 06**: Implement RAG with embedding APIs